In [1]:
# One-step ahead linear regression.  Does regression from full activity to full activity, 
# full activity to CD, and CD to CD. This code is currently only considering the late delay period (post-perturbation).

from __future__ import print_function
import argparse
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np
import os
import shutil
import time
import math
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from scipy.stats import ttest_rel
from sklearn.linear_model import RidgeCV, LassoCV, MultiTaskLassoCV
import pickle

os.environ["CUDA_VISIBLE_DEVICES"] = "0,"

from lfads_datasets import DualALMSimpleDataset

cwd = '..'




filenames = ['../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC4/BAYLORGC4_2018_03_22.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC4/BAYLORGC4_2018_03_21.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC4/BAYLORGC4_2018_03_23.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC4/BAYLORGC4_2018_03_20.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC4/BAYLORGC4_2018_03_25.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC12/BAYLORGC12_2018_04_03.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC12/BAYLORGC12_2018_04_04.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC12/BAYLORGC12_2018_04_05.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC12/BAYLORGC12_2018_04_06.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC13/BAYLORGC13_2018_04_23.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC13/BAYLORGC13_2018_04_24.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC13/BAYLORGC13_2018_04_25.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC15/BAYLORGC15_2018_05_07.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC15/BAYLORGC15_2018_05_08.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC15/BAYLORGC15_2018_05_09.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC15/BAYLORGC15_2018_05_10.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC15/BAYLORGC15_2018_05_11.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC17/BAYLORGC17_2018_06_08.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC17/BAYLORGC17_2018_06_11.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC18/BAYLORGC18_2018_05_31.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC19/BAYLORGC19_2018_06_20.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC19/BAYLORGC19_2018_06_21.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC19/BAYLORGC19_2018_06_22.mat']

savenames = ['BAYLORGC4_2018_03_22.mat', 'BAYLORGC4_2018_03_21.mat', 'BAYLORGC4_2018_03_23.mat', 'BAYLORGC4_2018_03_20.mat', 'BAYLORGC4_2018_03_25.mat', 'BAYLORGC12_2018_04_03.mat', 'BAYLORGC12_2018_04_04.mat', 'BAYLORGC12_2018_04_05.mat', 'BAYLORGC12_2018_04_06.mat', 'BAYLORGC13_2018_04_23.mat', 'BAYLORGC13_2018_04_24.mat', 'BAYLORGC13_2018_04_25.mat', 'BAYLORGC15_2018_05_07.mat', 'BAYLORGC15_2018_05_08.mat', 'BAYLORGC15_2018_05_09.mat', 'BAYLORGC15_2018_05_10.mat', 'BAYLORGC15_2018_05_11.mat', 'BAYLORGC17_2018_06_08.mat', 'BAYLORGC17_2018_06_11.mat', 'BAYLORGC18_2018_05_31.mat', 'BAYLORGC19_2018_06_20.mat', 'BAYLORGC19_2018_06_21.mat', 'BAYLORGC19_2018_06_22.mat']




for timebins in ['']: #'first' and 'last' let you do regression only on the first/second half of the late delay period
    for trainstimtype in ['']: #choose 'uni', 'early delay bi ALM', or 'no stim' for unilateral pert, bilateral pert, or control trials

        #Nomenclature: "nobridge" means independent predictions on each hemisphere.  "bridge" means using activity
        #from both hemisphere to make predictions.  "XToY" means using X to make predictions about Y -- so for example
        # "FullTCD" means predicting CD activity at time t+1 from full activity at time t
        alpha_valsFullToCD = []
        alpha_valsCDToCD = []
        alpha_valsFullToFull = []
        ratio_valsFullToCD = []
        ratio_valsCDToCD = []
        ratio_valsFullToFull = []
        
        bridgeFullToCD = []
        nobridgeFullToCD = []
        bridgeCDToCD = []
        nobridgeCDToCD = []
        bridgeFullToFull = []
        nobridgeFullToFull = []
        bridgeFullToCDPredTRAINL = []
        bridgeCDToCDPredTRAINL = []
        bridgeFullToFullPredTRAINL = []
        bridgeFullToCDPredTESTL = []
        bridgeCDToCDPredTESTL = []
        bridgeFullToFullPredTESTL = []

        bridgeFullToCDPredTRAINR = []
        bridgeCDToCDPredTRAINR = []
        bridgeFullToFullPredTRAINR = []
        bridgeFullToCDPredTESTR = []
        bridgeCDToCDPredTESTR = []
        bridgeFullToFullPredTESTR = []

        nobridgeFullToCDPredTRAINL = []
        nobridgeFullToCDPredTRAINR = []
        nobridgeCDToCDPredTRAINL = []
        nobridgeCDToCDPredTRAINR = []
        nobridgeFullToFullPredTRAINL = []
        nobridgeFullToFullPredTRAINR = []
        nobridgeFullToCDPredTESTL = []
        nobridgeFullToCDPredTESTR = []
        nobridgeCDToCDPredTESTL = []
        nobridgeCDToCDPredTESTR = []
        nobridgeFullToFullPredTESTL = []
        nobridgeFullToFullPredTESTR = []
        
        
        for indexX in range(len(filenames)):
            filename = filenames[indexX]
            savename = savenames[indexX]
            save_preds = ''
            if timebins == '':
                bins = range(8)
            elif timebins == 'firstbins':
                bins = range(4)
            elif timebins == 'lastbins':
                bins = range(4, 8)


            #loads the information about which trials are which stim type
            #only important if you have chosen to do the regression on a subset of trials, selected by stim type
            pred_file = 'OneStepModel' + str(1.0) + '_' + str('attractor_normalbridge') + '_' + str(timebins)+str(trainstimtype)+str(0.001)+savename + '_'
            stimtype_file = 'XRegSuperSimple' + str(1.0) + '_' + '_' + str(timebins)+str(trainstimtype)+ savename + '_'

            train_stimtypes = np.load(cwd+'/stimtypes/'+stimtype_file + 'TRAIN_stimtypes.npy')
            test_stimtypes = np.load(cwd+'/stimtypes/'+stimtype_file + 'TEST_stimtypes.npy')

            if trainstimtype == '':
                trainstimfilter = train_stimtypes < 100
            elif trainstimtype == 'uni':
                trainstimfilter = (train_stimtypes % 3) != 0 
            elif trainstimtype == 'early delay bi ALM':
                trainstimfilter = train_stimtypes == 3
            elif trainstimtype == 'no stim':
                trainstimfilter = train_stimtypes == 0

            if trainstimtype == '':
                teststimfilter = test_stimtypes < 100
            elif trainstimtype == 'uni':
                teststimfilter = (test_stimtypes % 3) != 0 
            elif trainstimtype == 'early delay bi ALM':
                teststimfilter = test_stimtypes == 3
            elif trainstimtype == 'no stim':
                teststimfilter = test_stimtypes == 0
                
            #loads and preprocesses the data.  This takes some time so I have saved the preprocessed version for
            #fast loading.  However if you want to mess around with the data preprocessing, choice of time bins, etc.
            # you will want to edit the DualALMSimpleDataset class and uncomment + rerun these lines
            
            #train_dataset = DualALMSimpleDataset(train=True, neuron_split=1.0, location=None, filename=filename, train_stim_type=trainstimtype, timebins=timebins)
            #test_dataset = DualALMSimpleDataset(train=False, neuron_split=1.0, location=None, filename=filename, train_stim_type=trainstimtype, timebins=timebins)

            #filehandler = open(args.filename+'ALLEVERYTHING_train_dataset.obj', 'wb')
            #pickle.dump(train_dataset, filehandler)
            #filehandler = open(args.filename+'ALLEVERYTHING_test_dataset.obj', 'wb')
            #pickle.dump(test_dataset, filehandler
            filehandler = open(cwd+'/OneStepPreprocessed/'+savename+'ALLEVERYTHING_train_dataset.obj', 'rb')
            train_dataset = pickle.load(filehandler)
            filehandler = open(cwd+'/OneStepPreprocessed/'+savename+'ALLEVERYTHING_test_dataset.obj', 'rb')
            test_dataset = pickle.load(filehandler)

            train_data = train_dataset.data.reshape(8, -1, train_dataset.data.shape[1])[bins][:, trainstimfilter].reshape(-1, train_dataset.data.shape[1])
            train_CDdata = train_dataset.CDdata.reshape(8, -1, 2)[bins][:, trainstimfilter].reshape(-1, 2)
            train_data_L = train_data[:, train_dataset.pa.train_data.neural_unit_location=='left_ALM']
            train_data_R = train_data[:, train_dataset.pa.train_data.neural_unit_location=='right_ALM']
            train_data_CDL = train_CDdata[:, 0:1]
            train_data_CDR = train_CDdata[:, 1:2]
            train_labels = train_dataset.labels.reshape(8, -1, 2)[bins][:, trainstimfilter].reshape(-1, 2)
            train_fulllabels = train_dataset.fulllabels.reshape(8, -1, train_dataset.data.shape[1])[bins][:, trainstimfilter].reshape(-1, train_dataset.data.shape[1])
            train_labels_L = train_labels[:, 0]
            train_labels_R = train_labels[:, 1]
            train_labels_fullL = train_fulllabels[:, train_dataset.pa.train_data.neural_unit_location=='left_ALM']
            train_labels_fullR = train_fulllabels[:, train_dataset.pa.train_data.neural_unit_location=='right_ALM']
            test_data = test_dataset.data.reshape(8, -1, test_dataset.data.shape[1])[bins][:, teststimfilter].reshape(-1, test_dataset.data.shape[1])
            test_CDdata = test_dataset.CDdata.reshape(8, -1, 2)[bins][:, teststimfilter].reshape(-1, 2)
            test_data_L = test_data[:, train_dataset.pa.train_data.neural_unit_location=='left_ALM']
            test_data_R = test_data[:, train_dataset.pa.train_data.neural_unit_location=='right_ALM']
            test_data_CDL = test_CDdata[:, 0:1]
            test_data_CDR = test_CDdata[:, 1:2]
            test_labels = test_dataset.labels.reshape(8, -1, 2)[bins][:, teststimfilter].reshape(-1, 2)
            test_fulllabels = test_dataset.fulllabels.reshape(8, -1, test_dataset.data.shape[1])[bins][:, teststimfilter].reshape(-1, test_dataset.data.shape[1])
            test_labels_L = test_labels[:, 0]
            test_labels_R = test_labels[:, 1]
            test_labels_fullL = test_fulllabels[:, train_dataset.pa.train_data.neural_unit_location=='left_ALM']
            test_labels_fullR = test_fulllabels[:, train_dataset.pa.train_data.neural_unit_location=='right_ALM']

            
            #Do the regressions.  the "alphas" parameter gives the range of L2 regularization factors to try
            regr = RidgeCV(alphas=np.arange(0.001, 1.0, 0.001))
            regr.fit(train_data, train_labels)
            loss = np.sum(np.square(test_labels - regr.predict(test_data)))
            bridgeFullToCDPredTRAINL.append(regr.predict(train_data)[:, 0:1])
            bridgeFullToCDPredTESTL.append(regr.predict(test_data)[:, 0:1])

            bridgeFullToCDPredTRAINR.append(regr.predict(train_data)[:, 1:2])
            bridgeFullToCDPredTESTR.append(regr.predict(test_data)[:, 1:2])
            alpha_valsFullToCD.append(regr.alpha_)
            #ratio_valsFullToCD.append(regr.l1_ratio)
            print(regr.alpha_)

            regr = RidgeCV(alphas=np.arange(0.001, 1.0, 0.001))
            regr.fit(train_data_L, train_labels_L)
            loss_L = np.sum(np.square(test_labels_L - regr.predict(test_data_L)))
            train_pred_L = regr.predict(train_data_L)
            test_pred_L = regr.predict(test_data_L)
            regr = RidgeCV(alphas=np.arange(0.001, 1.0, 0.001))
            regr.fit(train_data_R, train_labels_R)
            loss_R = np.sum(np.square(test_labels_R - regr.predict(test_data_R)))
            train_pred_R = regr.predict(train_data_R)
            test_pred_R = regr.predict(test_data_R)
            nobridgeFullToCDPredTRAINL.append(train_pred_L)
            nobridgeFullToCDPredTESTL.append(test_pred_L)
            nobridgeFullToCDPredTRAINR.append(train_pred_R)
            nobridgeFullToCDPredTESTR.append(test_pred_R)

            #the "baseline" for variance explained figures is just the identity transformation
            baseline = np.sum(np.square(test_labels - test_CDdata))
            print(1 - loss/baseline, 1 - (loss_L + loss_R)/baseline)
            #print(regr.alpha_)

            bridgeFullToCD.append(1 - loss/baseline)
            nobridgeFullToCD.append(1 - (loss_L + loss_R)/baseline)


            regr = RidgeCV(alphas=np.arange(0.001, 1.0, 0.001))
            regr.fit(train_CDdata, train_labels)
            loss = np.sum(np.square(test_labels - regr.predict(test_CDdata)))

            bridgeCDToCDPredTRAINL.append(regr.predict(train_CDdata)[:, 0:1])
            bridgeCDToCDPredTESTL.append(regr.predict(test_CDdata)[:, 0:1])

            bridgeCDToCDPredTRAINR.append(regr.predict(train_CDdata)[:, 1:2])
            bridgeCDToCDPredTESTR.append(regr.predict(test_CDdata)[:, 1:2])
            alpha_valsCDToCD.append(regr.alpha_)
            #ratio_valsCDToCD.append(regr.l1_ratio)
            print(regr.alpha_)

            regr = RidgeCV(alphas=np.arange(0.001, 1.0, 0.001))
            regr.fit(train_data_CDL, train_labels_L)
            loss_L = np.sum(np.square(test_labels_L - regr.predict(test_data_CDL)))
            train_pred_L = regr.predict(train_data_CDL)
            test_pred_L = regr.predict(test_data_CDL)
            regr = RidgeCV(alphas=np.arange(0.001, 1.0, 0.001))
            regr.fit(train_data_CDR, train_labels_R)
            loss_R = np.sum(np.square(test_labels_R - regr.predict(test_data_CDR)))
            train_pred_R = regr.predict(train_data_CDR)
            test_pred_R = regr.predict(test_data_CDR)
            nobridgeCDToCDPredTRAINL.append(train_pred_L)
            nobridgeCDToCDPredTESTL.append(test_pred_L)
            nobridgeCDToCDPredTRAINR.append(train_pred_R)
            nobridgeCDToCDPredTESTR.append(test_pred_R)
            baseline = np.sum(np.square(test_labels - test_CDdata))
            print(1 - loss/baseline, 1 - (loss_L + loss_R)/baseline)
            #print(regr.alpha_)

            bridgeCDToCD.append(1 - loss/baseline)
            nobridgeCDToCD.append(1 - (loss_L + loss_R)/baseline)

            regr = RidgeCV(alphas=np.arange(0.001, 1.0, 0.001))
            regr.fit(train_data, train_fulllabels)
            loss = np.sum(np.square(test_fulllabels - regr.predict(test_data)))

            bridgeFullToFullPredTRAINL.append(regr.predict(train_data)[:, train_dataset.pa.train_data.neural_unit_location=='left_ALM'])
            bridgeFullToFullPredTESTL.append(regr.predict(test_data)[:, train_dataset.pa.train_data.neural_unit_location=='left_ALM'])

            bridgeFullToFullPredTRAINR.append(regr.predict(train_data)[:, train_dataset.pa.train_data.neural_unit_location=='right_ALM'])
            bridgeFullToFullPredTESTR.append(regr.predict(test_data)[:, train_dataset.pa.train_data.neural_unit_location=='right_ALM'])

            alpha_valsFullToFull.append(regr.alpha_)
            #ratio_valsFullToFull.append(regr.l1_ratio)
            print(regr.alpha_)

            regr = RidgeCV(alphas=np.arange(0.001, 1.0, 0.001))
            regr.fit(train_data_L, train_labels_fullL)
            loss_L = np.sum(np.square(test_labels_fullL - regr.predict(test_data_L)))
            train_pred_L = regr.predict(train_data_L)
            test_pred_L = regr.predict(test_data_L)
            regr = RidgeCV(alphas=np.arange(0.001, 1.0, 0.001))
            regr.fit(train_data_R, train_labels_fullR)
            loss_R = np.sum(np.square(test_labels_fullR - regr.predict(test_data_R)))
            train_pred_R = regr.predict(train_data_R)
            test_pred_R = regr.predict(test_data_R)
            nobridgeFullToFullPredTRAINL.append(train_pred_L)
            nobridgeFullToFullPredTESTL.append(test_pred_L)
            nobridgeFullToFullPredTRAINR.append(train_pred_R)
            nobridgeFullToFullPredTESTR.append(test_pred_R)
            baseline = np.sum(np.square(test_fulllabels - test_data))
            print(1 - loss/baseline, 1 - (loss_L + loss_R)/baseline)
            #print(regr.alpha_)

            bridgeFullToFull.append(1 - loss/baseline)
            nobridgeFullToFull.append(1 - (loss_L + loss_R)/baseline)
        
        #saves the variance explained for each session, full activity to full activity regression
        np.save('XbridgeFullToFull'+timebins+trainstimtype, np.squeeze(bridgeFullToFull))
        
        #saves the full-to-full regresssion predictions for each session on the train/test set and left/right hemispheres
        np.save(cwd+'/SavedPreds/'+'XbridgeFullToFullPredTRAINL'+timebins+trainstimtype, np.squeeze(bridgeFullToFullPredTRAINL))
        np.save(cwd+'/SavedPreds/'+'XbridgeFullToFullPredTESTL'+timebins+trainstimtype, np.squeeze(bridgeFullToFullPredTESTL))
        np.save(cwd+'/SavedPreds/'+'XbridgeFullToFullPredTRAINR'+timebins+trainstimtype, np.squeeze(bridgeFullToFullPredTRAINR))
        np.save(cwd+'/SavedPreds/'+'XbridgeFullToFullPredTESTR'+timebins+trainstimtype, np.squeeze(bridgeFullToFullPredTESTR))
        
        np.save(cwd+'/SavedPreds/'+'XnobridgeFullToFull'+timebins+trainstimtype, np.squeeze(nobridgeFullToFull))
        np.save(cwd+'/SavedPreds/'+'XnobridgeFullToFullPredTRAINL'+timebins+trainstimtype, np.squeeze(nobridgeFullToFullPredTRAINL))
        np.save(cwd+'/SavedPreds/'+'XnobridgeFullToFullPredTESTL'+timebins+trainstimtype, np.squeeze(nobridgeFullToFullPredTESTL))
        np.save(cwd+'/SavedPreds/'+'XnobridgeFullToFullPredTRAINR'+timebins+trainstimtype, np.squeeze(nobridgeFullToFullPredTRAINR))
        np.save(cwd+'/SavedPreds/'+'XnobridgeFullToFullPredTESTR'+timebins+trainstimtype, np.squeeze(nobridgeFullToFullPredTESTR))
        
        np.save(cwd+'/SavedPreds/'+'XbridgeFullToCD'+timebins+trainstimtype, np.squeeze(bridgeFullToCD))
        np.save(cwd+'/SavedPreds/'+'XbridgeFullToCDPredTRAINL'+timebins+trainstimtype, np.squeeze(bridgeFullToCDPredTRAINL))
        np.save(cwd+'/SavedPreds/'+'XbridgeFullToCDPredTESTL'+timebins+trainstimtype, np.squeeze(bridgeFullToCDPredTESTL))
        np.save(cwd+'/SavedPreds/'+'XbridgeFullToCDPredTRAINR'+timebins+trainstimtype, np.squeeze(bridgeFullToCDPredTRAINR))
        np.save(cwd+'/SavedPreds/'+'XbridgeFullToCDPredTESTR'+timebins+trainstimtype, np.squeeze(bridgeFullToCDPredTESTR))
        
        np.save(cwd+'/SavedPreds/'+'XnobridgeFullToCD'+timebins+trainstimtype, np.squeeze(nobridgeFullToCD))
        np.save(cwd+'/SavedPreds/'+'XnobridgeFullToCDPredTRAINL'+timebins+trainstimtype, np.squeeze(nobridgeFullToCDPredTRAINL))
        np.save(cwd+'/SavedPreds/'+'XnobridgeFullToCDPredTESTL'+timebins+trainstimtype, np.squeeze(nobridgeFullToCDPredTESTL))
        np.save(cwd+'/SavedPreds/'+'XnobridgeFullToCDPredTRAINR'+timebins+trainstimtype, np.squeeze(nobridgeFullToCDPredTRAINR))
        np.save(cwd+'/SavedPreds/'+'XnobridgeFullToCDPredTESTR'+timebins+trainstimtype, np.squeeze(nobridgeFullToCDPredTESTR))
        
        np.save(cwd+'/SavedPreds/'+'XbridgeCDToCD'+timebins+trainstimtype, np.squeeze(bridgeCDToCD))
        np.save(cwd+'/SavedPreds/'+'XbridgeCDToCDPredTRAINL'+timebins+trainstimtype, np.squeeze(bridgeCDToCDPredTRAINL))
        np.save(cwd+'/SavedPreds/'+'XbridgeCDToCDPredTESTL'+timebins+trainstimtype, np.squeeze(bridgeCDToCDPredTESTL))
        np.save(cwd+'/SavedPreds/'+'XbridgeCDToCDPredTRAINR'+timebins+trainstimtype, np.squeeze(bridgeCDToCDPredTRAINR))
        np.save(cwd+'/SavedPreds/'+'XbridgeCDToCDPredTESTR'+timebins+trainstimtype, np.squeeze(bridgeCDToCDPredTESTR))
        
        np.save(cwd+'/SavedPreds/'+'XnobridgeCDToCDPredTRAINL'+timebins+trainstimtype, np.squeeze(nobridgeCDToCDPredTRAINL))
        np.save(cwd+'/SavedPreds/'+'XnobridgeCDToCDPredTESTL'+timebins+trainstimtype, np.squeeze(nobridgeCDToCDPredTESTL))
        np.save(cwd+'/SavedPreds/'+'XnobridgeCDToCDPredTRAINR'+timebins+trainstimtype, np.squeeze(nobridgeCDToCDPredTRAINR))
        np.save(cwd+'/SavedPreds/'+'XnobridgeCDToCDPredTESTR'+timebins+trainstimtype, np.squeeze(nobridgeCDToCDPredTESTR))
        np.save(cwd+'/SavedPreds/'+'XnobridgeCDToCD'+timebins+trainstimtype, np.squeeze(nobridgeCDToCD))
        
        #saves the optimal L2 regularization parameters for each session and each regression type
        np.save(cwd+'/SavedPreds/'+'XalphavalsFullToFull'+timebins+trainstimtype, alpha_valsFullToFull)
        np.save(cwd+'/SavedPreds/'+'XalphavalsFullToCD'+timebins+trainstimtype, alpha_valsFullToCD)
        np.save(cwd+'/SavedPreds/'+'XalphavalsCDToCD'+timebins+trainstimtype, alpha_valsCDToCD)
        


     

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


0.378
0.06961441922106126 0.03746760534720084
0.005
0.05348454172887296 0.025167252136048646


KeyboardInterrupt: 

In [14]:
import torch.utils.data as data
import torch
import torch.nn as nn
import numpy as np
import os
import sys
os.getcwd()
os.chdir('../../Code/Python')
sys.path.append('')

from neupop import Session as Session
from neupop import PopulationAnalysis as PA
from neupop import MultiSessionAnalysis as MS
from neupop import decoders as ld
from neupop import session_parsers as parsers

class DualALMSimpleDataset(data.Dataset):
    def __init__(self, train, neuron_split=1.0, back_half_neurons=False, normalize=True, location='left_ALM', train_stim_type='no stim', test_stim_type=None, filename='../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC12/BAYLORGC12_2018_04_04.mat', only_correct=True, timebins='', indexX = 0):
        # Determine normalizing factor, which is the maximum value across both train and test set.

        sd = Session(filename, parser=parsers.ParserNuoLiDualALM)
        self.train = train
        pa = PA(sd)

        (start, end) = pa.score_input_trial_range()
        pa.cull_data(start, end)
        if train:
            pa.load_train_and_test(train_stim_type=train_stim_type, test_stim_type=None, train_proportion=0.8, location=location)
        else:
            pa.load_train_and_test(train_stim_type=train_stim_type, test_stim_type=None, train_proportion=0.8, location=location)

        if only_correct:
            correct_mask = pa.input_data.behavior_report == 1
            self.fraction_correct = np.sum(pa.train_data.behavior_report) / len(pa.train_data.behavior_report)
            #print(filename, train_stim_type, location, train)
            #print('Frac correct', self.fraction_correct)
            self.behavior_report = pa.train_data.behavior_report
            if True:#train_stim_type != 'early delay bi ALM':
                pa.filter_train_trials_from_input(correct_mask)
                pa.filter_test_trials_from_input(correct_mask)

        pa.preprocess(bin_width=0.4, label_by_report=True, even_test_classes=False)

        train_data = pa.train_rates
        test_data = pa.test_rates

        if train:
            self.norm_factor = np.amax(train_data)
        else:
            self.norm_factor = max(np.amax(train_data), np.amax(test_data))
        self.train_label = pa.train_label
        self.test_label = pa.test_label

        if not normalize:
            self.norm_factor = 1


        self.pa = pa
        if True:#train_stim_type == 'no stim':
            self.coding_directions = [pa.compute_CD_for_each_bin()[0][i].W for i in range(len(pa.compute_CD_for_each_bin()[0]))]

        if train:
            self.data = train_data / self.norm_factor
            self.task_trial_type= pa.train_data.task_trial_type
            self.pole_on_time = np.mean(pa.train_data.task_pole_on_time)
            self.pole_off_time = np.mean(pa.train_data.task_pole_off_time)
            self.cue_on_time = np.mean(pa.train_data.task_cue_on_time)
            self.behavior_report_type = pa.train_data.behavior_report_type

  
 
        else:
            self.data = test_data / self.norm_factor
            self.task_trial_type= pa.test_data.task_trial_type
            self.pole_on_time = np.mean(pa.test_data.task_pole_on_time)
            self.pole_off_time = np.mean(pa.test_data.task_pole_off_time)
            self.cue_on_time = np.mean(pa.test_data.task_cue_on_time)
            self.behavior_report_type = pa.test_data.behavior_report_type

        #self.data = self.data.astype(float)
        #print('data shape', self.data.shape)
        if back_half_neurons:
            self.data = self.data[:, :, (int)(neuron_split*self.data.shape[2]):]
        else:
            self.data = self.data[:, :, :(int)(neuron_split*self.data.shape[2])]
        if timebins == '':
            times = [28, 29, 30, 31, 32, 33, 34, 35]
        elif timebins == 'firstbins':
            times = [28, 29, 30, 31]
        elif timebins == 'lastbins':
            times = [32, 33, 34, 35]
        labeltimes = [time + 1 for time in times]
        num_neurons = self.data.shape[2]
        datatemp = self.data
        self.data = datatemp[times, :, :].reshape(-1, num_neurons) #data is full activityat time t
        #print('data shape', self.data.shape)
        self.labels = datatemp[labeltimes, :, :].reshape(-1, num_neurons) #temp, to be changed later in the code

        self.cd_L = np.expand_dims(self.coding_directions[36][pa.train_data.neural_unit_location=='left_ALM'], 1)
        self.cd_R = np.expand_dims(self.coding_directions[36][pa.train_data.neural_unit_location=='right_ALM'], 1)
        self.cd_L /= np.linalg.norm(self.cd_L)
        self.cd_R /= np.linalg.norm(self.cd_R)
        self.fulllabels = self.labels #fulllabels are full activity at time t+1
        labelsL = np.matmul(self.labels[:, pa.train_data.neural_unit_location=='left_ALM'], self.cd_L)
        labelsR = np.matmul(self.labels[:, pa.train_data.neural_unit_location=='right_ALM'], self.cd_R)
        dataL = np.matmul(self.data[:, pa.train_data.neural_unit_location=='left_ALM'], self.cd_L)
        dataR = np.matmul(self.data[:, pa.train_data.neural_unit_location=='right_ALM'], self.cd_R)
        self.CDdata = np.concatenate([dataL, dataR], 1) #CDdata is CD activity at time t
        self.labels = np.concatenate([labelsL, labelsR], 1) # labels are CD activity at time t+1

        linreg_file = 'bridgeFullToCDPred'
        
        if train:
            self.linreg_L = np.squeeze(np.load(linreg_file+'TRAINL'+'.npy')[indexX])#[:, :, :]#[:, 13:, :]
            self.linreg_R = np.squeeze(np.load(linreg_file+'TRAINR'+'.npy')[indexX])#[:, :, :]#[:, 13:, :]
        else:
            self.linreg_L = np.squeeze(np.load(linreg_file+'TESTL'+'.npy')[indexX])#[:, :, :]#[:, 13:, :]
            self.linreg_R = np.squeeze(np.load(linreg_file+'TESTR'+'.npy')[indexX])#[:, :, :]#[:, 13:, :]

        linreg_file = 'nobridgeFullToCDPred'
        
        if train:
            self.nobridge_linreg_L = np.squeeze(np.load(linreg_file+'TRAINL'+'.npy')[indexX])#[:, :, :]#[:, 13:, :]
            self.nobridge_linreg_R = np.squeeze(np.load(linreg_file+'TRAINR'+'.npy')[indexX])#[:, :, :]#[:, 13:, :]
        else:
            self.nobridge_linreg_L = np.squeeze(np.load(linreg_file+'TESTL'+'.npy')[indexX])#[:, :, :]#[:, 13:, :]
            self.nobridge_linreg_R = np.squeeze(np.load(linreg_file+'TESTR'+'.npy')[indexX])#[:, :, :]#[:, 13:, :]
        

        #assert self.data.shape[1] == self.labels.shape[1]

    def get_data_max(self):
        return np.amax(self.data)

    def get_data_min(self):
        return np.amin(self.data)

    def get_norm_factor(self):
        return self.norm_factor

    def __getitem__(self, index):
        # Note that for sequence datasets like this, data has shape (seq_len, n_data, input_size).
        #print('My index is ', index)
        x = self.data[index,:]
        y = self.labels[index, :]
        linreg_L = self.linreg_L[index]
        linreg_R = self.linreg_R[index]
        nobridge_linreg_L = self.nobridge_linreg_L[index]
        nobridge_linreg_R = self.nobridge_linreg_R[index]
        return x, y, linreg_L, linreg_R, nobridge_linreg_L, nobridge_linreg_R


    def __len__(self):
        # Note that for sequence datasets like this, data has shape (seq_len, n_data, input_size).
        return self.data.shape[0]

    def get_len(self):
        # Note that for sequence datasets like this, data has shape (seq_len, n_data, input_size).
        return self.data.shape[0]